In [1]:
# Staat helemaal bovenin assignment onder part 1
import numpy as np
import matplotlib.pyplot as plt
import math
import time
import random

from scipy.stats import norm
from numba import jit
from scipy import stats

# Part 2: Estimation of Sensitivities in MC

## 1) Bump-and-revalue method.

In [38]:
@jit (nopython = True)
def one_step_stock(stock_price, interest_rate, volatility, maturity, Z):
#     np.random.seed(5)
    return stock_price * math.exp((interest_rate - 0.5 * volatility ** 2) * maturity + volatility * math.sqrt(maturity) * Z)

@jit (nopython = True)
def monte_carlo_option(trials, strike_price, stock_price, interest_rate, volatility, maturity, randomness):
    # trials are #iterations of monte carlo
    monte_results = []
    for i in range(trials):
        current_monte = one_step_stock(stock_price, interest_rate, volatility, maturity, randomness[i])
        if current_monte - strike_price < 0:
            monte_results.append((strike_price - current_monte)/ (1 + interest_rate))
        else:
            monte_results.append(0)
        
    return monte_results 

In [59]:
@jit (nopython = True)
def iters(sims):
    lege = [] 
    T = 1
    K = 99
    r = 0.06
    S0 = 100 
    sigma = 0.2
    randomness =  np.random.normal(size=int(100000))
    for i in range(100000):
        iets = monte_carlo_option(100000, K, S0, r, sigma, T, randomness)
        lege.append(iets)
    return np.mean(lege)


In [ ]:
# lege = [] 
# T = 1
# K = 99
# r = 0.06
# S0 = 100 
# sigma = 0.2
# randomness =  np.random.normal(size=10000)
# for i in range(10000):
#     iets = monte_carlo_option(10000, K, S0, r, sigma, T, randomness)
#     lege.append(iets)

# print(np.mean(lege))

print(iters(10000))

In [39]:
def experiment(epsilon, randommess, trials ): #= 1000
    ''' The Monte Carlo method.'''
    T = 1
    K = 99
    r = 0.06
    S0 = 100 + epsilon
    sigma = 0.2

#     time0 = time.time()

    monte_total = []
    iters = trials #= 1000
    for i in range(iters):
        monte_results = monte_carlo_option(trials, K, S0, r, sigma, T, randommess)
        # print(time.time() - time0)

        # Mean 
        monte_mean = np.mean(monte_results)
        monte_total.append(monte_mean)

    return monte_total

In [40]:
def euler(epsilon, trials, randommess, sameseed): #    randommess = np.random.normal(size=1000), sameseed = True
    bumped = np.mean(experiment(epsilon, randommess, trials))
    if not sameseed:
        randommess = np.random.normal(size=number_of_trials)
    unbumped = np.mean(experiment(0, randommess, trials))
    
    delta = (bumped - unbumped) / epsilon
    return delta

In [41]:
''' The analytical delta. '''
def analytical_delta(current_stock_price, strike_price, interest_rate, volatility, T, t, steps):
    d1 = (math.log(current_stock_price / strike_price) + (interest_rate + (volatility ** 2) / 2) * T) / (volatility * math.sqrt(T))
    return norm.cdf(d1) 

In [44]:
""" Relative error in percentage. """
def relative_error(true_value, approx_value):
    return abs(((true_value - approx_value) / true_value) * 100)

In [45]:
""" Relative error is shown with analytical value as reference. """

# The analytical value
# analytical_value = -0.326265
T = 1
K = 99
r = 0.06
S0 = 100 
sigma = 0.2

steps = 100
t = 0
check_analytical_value = analytical_delta(S0, K, r, sigma, T, t, steps) - 1
diff = abs(analytical_value - check_analytical_value)
print('check', analytical_value, check_analytical_value)
print('difference', diff)


# The approximate value 
# number_of_trials = int(10000) #1e4
# randommess = np.random.normal(size= number_of_trials)
# dif_delta4_1 = euler(0.01, trials = number_of_trials, randommess = randommess, sameseed=False) #1e-2

number_of_trials = int(1e4)
randommess = np.random.normal(size=number_of_trials)
same_delta4_1 = euler(1e-2, trials = number_of_trials, randommess = randommess, sameseed=True)

# print
print('analytical value:', analytical_value)
print('calculated value', same_delta4_1)
error = relative_error(analytical_value, same_delta4_1)
print('Error', error)

check -0.326265 -0.3262644882651039
difference 5.117348961247714e-07
analytical value: -0.326265
calculated value -0.3250724932886584
Error 0.3655024937831622


In [ ]:
# """ Compute deltas for different seeds
#     with epsilon 0.01, 0.02, 0.5. """

# # Size 10^4
# number_of_trials = int(1e4)
# randommess = np.random.normal(size= number_of_trials)
# dif_delta4_1 = euler(1e-2, trials = number_of_trials, randommess = randommess, sameseed=False)
# dif_delta4_2 = euler(2e-2, trials = number_of_trials, randommess = randommess, sameseed=False)
# dif_delta4_5 = euler(5e-1, trials = number_of_trials, randommess = randommess, sameseed=False)

# # Size 10^5
# number_of_trials = int(1e5)
# randommess = np.random.normal(size=number_of_trials)
# dif_delta5_1 = euler(1e-2, trials = number_of_trials, randommess = randommess, sameseed=False)
# dif_delta5_2 = euler(2e-2, trials = number_of_trials, randommess = randommess, sameseed=False)
# dif_delta5_5 = euler(5e-1, trials = number_of_trials, randommess = randommess, sameseed=False)

# # Size 10^6
# number_of_trials = int(1e6)
# randommess = np.random.normal(size=number_of_trials)
# dif_delta6_1 = euler(1e-2, trials = number_of_trials, randommess = randommess, sameseed=False)
# dif_delta6_2 = euler(2e-2, trials = number_of_trials, randommess = randommess, sameseed=False)
# dif_delta6_5 = euler(5e-1, trials = number_of_trials, randommess = randommess, sameseed=False)

# # Size 10^7
# number_of_trials = int(1e7)
# randommess = np.random.normal(size=number_of_trials)
# dif_delta7_1 = euler(1e-2, trials = number_of_trials, randommess = randommess, sameseed=False)
# dif_delta7_2 = euler(2e-2, trials = number_of_trials, randommess = randommess, sameseed=False)
# dif_delta7_5 = euler(5e-1, trials = number_of_trials, randommess = randommess, sameseed=False)

In [ ]:
# """ Compute deltas for same seed 
#     with epsilon 0.01, 0.02, 0.5. """

# # Size 10^4
# number_of_trials = int(1e4)
# randommess = np.random.normal(size=number_of_trials)
# same_delta4_1 = euler(1e-2, trials = number_of_trials, randommess = randommess, sameseed=True)
# same_delta4_2 = euler(2e-2, trials = number_of_trials, randommess = randommess, sameseed=True)
# same_delta4_5 = euler(5e-1, trials = number_of_trials, randommess = randommess, sameseed=True)

# # Size 10^5
# number_of_trials = int(1e5)
# randommess = np.random.normal(size=number_of_trials)
# same_delta5_1 = euler(1e-2, trials = number_of_trials, randommess = randommess, sameseed=True)
# same_delta5_2 = euler(2e-2, trials = number_of_trials, randommess = randommess, sameseed=True)
# same_delta5_5 = euler(5e-1, trials = number_of_trials, randommess = randommess, sameseed=True)

# # Size 10^6
# number_of_trials = int(1e6)
# randommess = np.random.normal(size=number_of_trials)
# same_delta6_1 = euler(1e-2, trials = number_of_trials, randommess = randommess, sameseed=True)
# same_delta6_2 = euler(2e-2, trials = number_of_trials, randommess = randommess, sameseed=True)
# same_delta6_5 = euler(5e-1, trials = number_of_trials, randommess = randommess, sameseed=True)

# # Size 10^7
# number_of_trials = int(1e7)
# randommess = np.random.normal(size=number_of_trials)
# same_delta7_1 = euler(1e-2, trials = number_of_trials, randommess = randommess, sameseed=True)
# same_delta7_2 = euler(2e-2, trials = number_of_trials, randommess = randommess, sameseed=True)
# same_delta7_5 = euler(5e-1, trials = number_of_trials, randommess = randommess, sameseed=True)